### 2021_10_15_4

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [5]:
total = pd.read_csv('../CSV/한국가스공사_시간별_공급량_20181231.csv', encoding='cp949')

In [6]:
total.head(3)

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505


### 데이터 전처리

In [7]:
total["구분"].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [8]:
d_map = {}
for i, d in enumerate(total["구분"].unique()) :
    d_map[d] = i
total["구분"] = total["구분"].map(d_map)

In [9]:
total.head(10)

,연월일,시간,구분,공급량
0,2013-01-01,1,0,2497.129
1,2013-01-01,2,0,2363.265
2,2013-01-01,3,0,2258.505
3,2013-01-01,4,0,2243.969
4,2013-01-01,5,0,2344.105
5,2013-01-01,6,0,2390.961
6,2013-01-01,7,0,2378.457
7,2013-01-01,8,0,2518.921
8,2013-01-01,9,0,2706.481
9,2013-01-01,10,0,2832.057


In [10]:
total["연월일"] = pd.to_datetime(total["연월일"])

In [11]:
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [12]:
train_years = [2013, 2014, 2015, 2016, 2017]
val_years = [2018]

In [13]:
train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

In [14]:
features = ['구분', 'month', 'day', 'weekday', '시간']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

### 학습

In [15]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}

model = lgb.train(params, d_train, 500, d_val, verbose_eval=20, early_stopping_rounds=10)
pred = model.predict(val_x)
train_mse = mean_squared_error(val_y, pred)
print("MSE :", train_mse)

# cross_val = cross_val_score(model, val_x, val_y, cv = 5,
#                            scoring = "neg_mean_squared_error")
# print("cross validation :", cross_val)


/Users/seungwoomun/opt/anaconda3/envs/likelion/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/seungwoomun/opt/anaconda3/envs/likelion/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001404 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 244.857
[40]	valid_0's l1: 174.855
[60]	valid_0's l1: 158.739
[80]	valid_0's l1: 153.323
[100]	valid_0's l1: 150.948
[120]	valid_0's l1: 150.463
Early stopping, best iteration is:
[112]	valid_0's l1: 150.297
MSE : 68453.67846858544


### 추론 및 결과 제출

In [16]:
test = pd.read_csv('../CSV/가스공급량 수요예측 모델개발 data/test.csv')
submission = pd.read_csv('../CSV/가스공급량 수요예측 모델개발 data/sample_submission.csv')

In [17]:
test.head(3)

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A


In [18]:
submission.head(3)

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0


In [19]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [20]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [21]:
test['구분'] = test['구분'].map(d_map)

In [22]:
test_x = test[features]

In [23]:
test_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
15115,6,3,31,6,20
15116,6,3,31,6,21
15117,6,3,31,6,22
15118,6,3,31,6,23


In [24]:
preds = model.predict(test_x)
submission['공급량'] = preds
# submission.to_csv('../CSV/가스공급량 수요예측 모델개발 data/baseline.csv', index = False)

In [25]:
def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

In [26]:
a = pd.DataFrame({"y" : val_y})
b = pd.DataFrame({"pred_y" : pred})

a.reset_index(drop = True, inplace = True)
a.reset_index(inplace = True)
b.reset_index(inplace = True)

print("NMAE :", nmae(a, b))

NMAE : 0.5161785945623046
